<a href="https://colab.research.google.com/github/qharo/googleColab/blob/master/recurrentNeuralNetwork.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%tensorflow_version 2.x
from tensorflow.keras.preprocessing import sequence
from tensorflow import keras
import tensorflow as tf
import os
import numpy as np

#GET THE FILE
path_to_file = keras.utils.get_file('shakespeare.txt', 'https://storage.googleapis.com/download.tensorflow.org/data/shakespeare.txt')
text = open(path_to_file, 'rb').read().decode(encoding='utf-8')

#ENCODING
vocab = sorted(set(text))
char2idx = {u:i for i,u in enumerate(vocab)}
idx2char = np.array(vocab)

def text_to_int(text):
    return np.array([char2idx[c] for c in text])

text_as_int = text_to_int(text)

#DECODING
def int_to_text(ints):
    try:
        ints = ints.numpy()
    except:
        pass
    return ' '.join(idx2char[ints])

#MOLDING THE TRAINING EXAMPLES
seq_length = 100
examples_per_epoch = len(text)//(seq_length+1)

char_dataset = tf.data.Dataset.from_tensor_slices(text_as_int)

sequences = char_dataset.batch(seq_length+1, drop_remainder=True)

#MAKING INPUT/OUTPUT DATA
def split_input_target(chunk):
    input_text = chunk[:-1]
    target_text = chunk[1:]
    return input_text, target_text

dataset = sequences.map(split_input_target)

#MAKE TRAINING BATCHES
BATCH_SIZE = 64
VOCAB_SIZE = len(vocab)
EMBEDDING_DIM = 256
RNN_UNITS = 1024

BUFFER_SIZE = 10000

data = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder = True)

def build_model(vocab_size, embedding_dim, rnn_units, batch_size):
    model = tf.keras.Sequential([
        keras.layers.Embedding(vocab_size, embedding_dim, batch_input_shape=[batch_size, None]),
        keras.layers.LSTM(rnn_units, return_sequences=True, stateful=True, recurrent_initializer="glorot_uniform"),
        keras.layers.Dense(vocab_size),
    ])
    return model

model = build_model(VOCAB_SIZE, EMBEDDING_DIM, RNN_UNITS, BATCH_SIZE)

#CREATING A LOSS FUNCTION
def loss(labels, logits):
    return keras.losses.sparse_categorical_crossentropy(labels, logits, from_logits=True)

#COMPILE MODEL
model.compile(optimizer='adam', loss=loss)

#CHECKPOINTS
checkpoint_dir = './training_checkpoints'
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")

checkpoint_callback = keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix, 
    save_weights_only=True
)

#TRAINING MODEL
history = model.fit(data, epochs=40, callbacks=[checkpoint_callback])

#LOADING MODEL
model = build_model(VOCAB_SIZE, EMBEDDING_DIM,RNN_UNITS, batch_size=1)
model.load_weights(tf.train.latest_checkpoint(checkpoint_dir))
model.build(tf.TensorShape([1,None]))
    

1122304/1115394 [==============================] - 0s 0us/step
Epoch 1/40
172/172 [==============================] - 9s 54ms/step - loss: 2.5968
Epoch 2/40
172/172 [==============================] - 9s 54ms/step - loss: 1.8751
Epoch 3/40
172/172 [==============================] - 9s 54ms/step - loss: 1.6324
Epoch 4/40
172/172 [==============================] - 9s 54ms/step - loss: 1.5030
Epoch 5/40
172/172 [==============================] - 9s 54ms/step - loss: 1.4236
Epoch 6/40
172/172 [==============================] - 9s 54ms/step - loss: 1.3677
Epoch 7/40
172/172 [==============================] - 9s 54ms/step - loss: 1.3237
Epoch 8/40
172/172 [==============================] - 9s 54ms/step - loss: 1.2854
Epoch 9/40
172/172 [==============================] - 9s 54ms/step - loss: 1.2504
Epoch 10/40
172/172 [==============================] - 9s 54ms/step - loss: 1.2167
Epoch 11/40
172/172 [==============================] - 9s 54ms/step - loss: 1.1815
Epoch 12/40
172/172 [============

In [2]:
#ACTUALLY GENERATING TEXT
def generate_text(model, start_string):
    num_generate = 800
    input_eval = [char2idx[s] for s in start_string]
    input_eval = tf.expand_dims(input_eval, 0)
    text_generated = []
    temmperature = 1.0
    model.reset_states()
    for i in range(num_generate):
        predictions = model(input_eval)
        predictions = tf.squeeze(predictions, 0)
        predictions = predictions/temmperature
        predicted_id = tf.random.categorical(predictions, num_samples=1)[-1,0].numpy()
        input_eval = tf.expand_dims([predicted_id], 0)
        text_generated.append(idx2char[predicted_id])
    return (start_string + ' '.join(text_generated))

inp = input("Type a starting string: ")
print(generate_text(model, inp))

Type a starting string: hello
helloa d . 
 
 T R A N I O : 
 S o f t !   t h o u   k n o w ' s t   n o   l a w   t o   m e   a r e   r e a s o n ' d . 
 
 B U C K I N G H A M : 
 U p o n   t h e   o ' e r l o o d ,   b e c a u s e   I   w i l l   b e 
 d a u g h t e r   i n   y o u r   h a n d : 
 O ,   h e   i s   c o m e ,   l i k e   o n e :   w e   m u s t   d e p a r t ; 
 B u t   f a r   i n   n e e d   w i l l   d r a w   y o u   n o   p o w e r ,   I   w o u l d 
 H e   l o n g   h a v e   I   w a t c h   t h e   a i r   d o c k ? 
 O   t h o u   b e l o v e d ?   o n l y   t h y   w i l l ! 
 
 K I N G   R I C H A R D   I I I : 
 A n d   f r o m   t h i s   i s l a n d ' s   i n s t r u c t i o n ,   h a n g   a   s u b j e c t , 
 A n d   I   a m   g o i n g   w i t h   i n s t r u c t i o n   t o   h i s . 
 A s   i f   y o u   s e e   a   k i n g ,   a n d   a j o t h e s ,   g r a c i o u s   V e r o n a ; 
 W h i c h   w e   i n   V i n c e n t i o .   W h i c h   t r i c